In [57]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from io import open
import glob
import os

import pandas as pd

from os import listdir
from PIL import Image as PImage

import matplotlib.pyplot as plt 
import matplotlib.image as img

import importlib

# from sam.sam import *
import time

In [132]:
def findFiles(path, rec=False): return glob.glob(path,recursive=rec) 

def get_id (value):
    keys = ['1', '2', '3', '3_hook', '4', '5', '6', '7', '8', 'a', 'b', 'b_nothumb', 
            'b_thumb', 'cbaby', 'obaby', 'by', 'c', 'd', 'e', 'f', 'f_open', 'fly', 
            'fly_nothumb', 'g', 'h', 'h_hook', 'h_thumb', 'i', 'jesus', 'k', 'l_hook', 
            'middle', 'm', 'n', 'o', 'index', 'index_flex', 'index_hook', 'pincet', 
            'ital', 'ital_thumb', 'ital_nothumb', 'ital_open', 'r', 's', 'write', 
            'spoon', 't', 'v', 'v_flex', 'v_hook', 'v_thumb', 'w', 'y', 'ae', 
            'ae_thumb', 'pincet_double', 'obaby_double', 'm2', 'jesus_thumb']
    return keys.index(value)+1

def getAllLabels():
    train = pd.read_csv(r'1miohands-v2-trainingalignment.txt', sep=" ", header=None, names=["path", "label"])
    train.path = train.path.apply(lambda x: x.replace('/work/cv2/koller/features/danish_nz_ph2014/hand.20151016/data/joint/','/Users/madina/Desktop/dataset/1miohands-v2-training/'))
    
    test = pd.read_csv(r'3359-ph2014-MS-handshape-annotations.txt', sep=" ", header=None, names=["path", "label"])
    test.path = test.path.apply(lambda x: x.replace('images/','/Users/madina/Desktop/dataset/1miohands-v2-training/'))
    test.label = test.label.apply(lambda x: get_id(x))

    return train, test

def getLabel(df, path):
    return df[df.path.str.contains(path)].label.iat[0]

In [133]:
import sys,glob
import os


def createLabels(allLabels, expectedDir):
    df = pd.DataFrame(columns=['path', 'label', 'name'])

    for fileName_relative in findFiles(expectedDir+"**/*.png", True):       
        fileName_absolute = os.path.basename(fileName_relative)
        df = df.append({'path': fileName_relative, 'label': getLabel(allLabels, fileName_relative), 'name': fileName_absolute}, ignore_index=True)
    return df
    
alTrain, alTest = getAllLabels()
trainExpectedDir = '/Users/madina/Desktop/dataset/1miohands-v2-training/danish_nz_ph2014/'
testExpectedDir = '/Users/madina/Desktop/dataset/1miohands-v2-training/final_phoenix_noPause_noCompound_lefthandtag_noClean'

train = createLabels(alTrain, trainExpectedDir)
test = createLabels(alTest, testExpectedDir)

train.to_csv(r'train.csv')
test.to_csv(r'test.csv')

KeyboardInterrupt: 

In [58]:
def findFiles(path): return glob.glob(path)

def get_id (value):
    keys = ['1', '2', '3', '3_hook', '4', '5', '6', '7', '8', 'a', 'b', 'b_nothumb', 
            'b_thumb', 'cbaby', 'obaby', 'by', 'c', 'd', 'e', 'f', 'f_open', 'fly', 
            'fly_nothumb', 'g', 'h', 'h_hook', 'h_thumb', 'i', 'jesus', 'k', 'l_hook', 
            'middle', 'm', 'n', 'o', 'index', 'index_flex', 'index_hook', 'pincet', 
            'ital', 'ital_thumb', 'ital_nothumb', 'ital_open', 'r', 's', 'write', 
            'spoon', 't', 'v', 'v_flex', 'v_hook', 'v_thumb', 'w', 'y', 'ae', 
            'ae_thumb', 'pincet_double', 'obaby_double', 'm2', 'jesus_thumb']
    return keys.index(value)+1

def createLabels(outputTrain, outputTest):
    train = pd.read_csv(r'1miohands-v2-trainingalignment.txt', sep=" ", header=None, names=["path", "label"])
    train.path = train.path.apply(lambda x: x.replace('/work/cv2/koller/features/danish_nz_ph2014/hand.20151016/data/joint/',''))
    train.to_csv(outputTrain)
    
    test = pd.read_csv(r'3359-ph2014-MS-handshape-annotations.txt', sep=" ", header=None, names=["path", "label"])
    test.path = test.path.apply(lambda x: x.replace('images/',''))
    test.label = test.label.apply(lambda x: get_id(x))
#     dups_color['label'] = df.apply (lambda row: get_id(row), axis=1)
#     dups_color = test.pivot_table(index=['label'], aggfunc='size')
    test.to_csv(outputTest)
    
    return train, test

train, test = createLabels(r'train.csv', r'test.csv')

In [59]:
mydict = {1: "1", 2: "2", 3: "3", 4: "3_hook", 5: "4", 
          6: "5", 7: "6", 8: "7", 9: "8", 10: "a",
          11: "b", 12: "b_nothumb", 13: "b_thumb", 14: "cbaby", 15: "obaby",
          16: "by", 17: "c", 18: "d", 19: "e", 20: "f",
          21: "f_open", 22: "fly", 23: "fly_nothumb", 24: "g", 25: "h",
          26: "h_hook", 27: "h_thumb", 28: "i", 29: "jesus", 30: "k",
          31: "l_hook", 32: "middle", 33: "m", 34: "n", 35: "o",
          36: "index", 37: "index_flex", 38: "index_hook", 39: "pincet", 40: "ital",
          41: "ital_thumb", 42: "ital_nothumb", 43: "ital_open", 44: "r", 45: "s",
          46: "write", 47: "spoon", 48: "t", 49: "v", 50: "v_flex",
          51: "v_hook", 52: "v_thumb", 53: "w", 54: "y", 55: "ae",
          56: "ae_thumb", 57: "pincet_double", 58: "obaby_double", 59: "m2", 60: "jesus_thumb"}
# mydict.values()
keys = ['1', '2', '3', '3_hook', '4', '5', '6', '7', '8', '1', 'b', 'b_nothumb', 'b_thumb', 'cbaby', 'obaby', 'by', 'c', 'd', 'e', 'f', 'f_open', 'fly', 'fly_nothumb', 'g', 'h', 'h_hook', 'h_thumb', 'i', 'jesus', 'k', 'l_hook', 'middle', 'm', 'n', 'o', 'index', 'index_flex', 'index_hook', 'pincet', 'ital', 'ital_thumb', 'ital_nothumb', 'ital_open', 'r', 's', 'write', 'spoon', 't', 'v', 'v_flex', 'v_hook', 'v_thumb', 'w', 'y', 'ae', 'ae_thumb', 'pincet_double', 'obaby_double', 'm2', 'jesus_thumb']
keys.index('b_nothumb')+1

12

In [51]:
import skimage
print(skimage.__version__)

0.13.0


In [60]:
# from skimage.io import imread

class Dataset(Dataset):
    def __init__(self, data , transform = None):
        self.data = data.values
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        img_name,label = self.data[index]
        img_path = os.path.join(img_name)
        image = PImage.open(img_path)

#         image = imread(img_name)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [56]:
def loadDatabase():
    
    train_labels = pd.read_csv(r'train.csv', index_col=[0])
    test_labels = pd.read_csv(r'test.csv', index_col=[0])

    train_data, validation_data = train_test_split(train_labels, stratify=train_labels.label, test_size=0.1)
    
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    valid_transform = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    test_transform = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    train_data = Dataset(train_data, train_transform )
    validation_data = Dataset(validation_data, valid_transform )
    test_data = Dataset(test_labels, test_transform )


    train_loader = DataLoader(train_data, batch_size=64, num_workers=0, shuffle=True)
    validation_loader = DataLoader(train_data, batch_size=64, num_workers=0, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64, num_workers=0, shuffle=True)
    
    return train_loader, validation_loader, test_loader

In [69]:
train_labels = pd.read_csv(r'train.csv', index_col=[0])
test_labels = pd.read_csv(r'test.csv', index_col=[0])
# train_labels

/Users/madina/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [70]:

train_data, validation_data = train_test_split(train_labels, test_size=0.1)


print(pd.concat([train_labels, test_labels]))

                                                   path  label
0     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
1     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
2     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
3     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
4     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
5     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
6     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
7     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
8     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
9     danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
10    danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
11    danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
12    danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
13    danish_nz_ph2014/01April_2010_Thursday_heute_d...      0
14    danish_nz_ph2014/01April_2010_Thursday_heute_d...

In [ ]:
class ResNetBasicBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding):
        super(ResNetBasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size, 1, padding)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
        x = F.relu(self.bn(self.conv1(x)))
        x = self.conv2(x)
        return x

class ResNetBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(ResNetBlock, self).__init__()
        self.in_ch = in_ch
        self.out_ch = out_ch   
        self.proj = nn.Conv2d(in_ch, out_ch, 1, 2)
        self.bn_proj = nn.BatchNorm2d(out_ch)
        
        if (in_ch == out_ch):
            self.block1 = ResNetBasicBlock(in_ch, out_ch, 3, 1, 1)
        else:
            self.block1 = ResNetBasicBlock(in_ch, out_ch, 3, 2, 1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.block2 = ResNetBasicBlock(out_ch, out_ch, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(out_ch)
        self.block3 = ResNetBasicBlock(out_ch, out_ch, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(out_ch)

    def forward(self, x):

        if (self.in_ch == self.out_ch):
            shortcut1 = x.clone()
        else:
            shortcut1 = self.proj(x)
        x = self.block1(x)
        x += shortcut1
        x = self.bn1(x)
        x = F.relu(x)

        shortcut2 = x.clone()
        x = self.block2(x)
        x += shortcut2
        x = self.bn2(x)
        x = F.relu(x)

        shortcut3 = x.clone()
        x = self.block3(x)
        x += shortcut3
        x = self.bn3(x)

        return x

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.bn0 = nn.BatchNorm2d(16)
        self.resBlock1 = ResNetBlock(16, 16)
        self.resBlock2 = ResNetBlock(16, 32)
        self.resBlock3 = ResNetBlock(32, 64)
        self.avgPool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.bn0(self.conv1(x)))

        x = F.relu(self.resBlock1(x))
        x = F.relu(self.resBlock2(x))
        x = F.relu(self.resBlock3(x))

        x = self.avgPool(x)
        x = x.view(-1, 64)
        x = self.fc(x)

        return x

In [ ]:
def train(model, device, train_loader, optimizer, cross_entropy, epoch, isSam):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
#         loss = F.nll_loss(output, target)
        loss = cross_entropy(output, target)
        loss.backward()
        
        optimizer.step()
        
#         if isSam:
#             optimizer.first_step(zero_grad=True)
#             cross_entropy(model(data), target).mean().backward()
#             optimizer.second_step(zero_grad=True)
#         else:
#             optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return loss.item()
       
def validate(model, device, validation_loader, cross_entropy):
    model.eval()
    valid_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
#             valid_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            valid_loss += cross_entropy(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    valid_loss /= len(validation_loader.dataset)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        valid_loss, correct, len(validation_loader.dataset),
        100. * correct / len(validation_loader.dataset)))
    return valid_loss

def test(model, device, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
            correct, total, 100. * correct / total))

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_loader, validation_loader, test_loader = loadDatabase(isData1=True)

model = ResNet().to(device)

cross_entropy = nn.CrossEntropyLoss()

adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.0001)
# sam_optimizer = SAM(model.parameters(), torch.optim.SGD, lr=0.001, momentum=0.9)
# adadelta_optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001)
# sgd_optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

counter, train_losses, valid_losses = [], [], []

for epoch in range(1, 50 + 1):

    train_losses.append( train(model, device, train_loader, adam_optimizer, cross_entropy, epoch, False) )
    valid_losses.append( validate(model, device, validation_loader, cross_entropy) )
    counter.append(epoch)

plt.figure(figsize=(9, 6))
plt.ylabel("Loss")
plt.xlabel("Number of Epochs")
plt.plot(counter, train_losses, "r", label = "Train loss")
plt.plot(counter, valid_losses, "b", label = "Validation loss")
plt.title("Loss")
plt.show()

torch.save(model.state_dict(), 'model1.ckpt') 

test(model, device, test_loader)